In [1]:
import os 
import shutil
import pandas as pd
import pathlib
from sklearn.model_selection import train_test_split
import shutil
import tensorflow as tf
import sys
sys.path.append('..')
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from training.Helpers import metrics,make_model_cv,load_cv_data,take_params,test_cv_model
from tqdm import tqdm

In [2]:
#dataprep

# desktop = pathlib.Path('../../pics_dataset/')
# ids = [ [x,int(x.name.split('_')[1])] for x in  list(desktop.rglob("*.png"))]
# data = pd.DataFrame(ids,columns=['path','id'])
# data
# y = pd.read_pickle('../data/datasets/train/coding_pic.pkl')
# x = pd.read_pickle('../data/datasets/train/data_40k.pkl')
# x = x.loc[x.index.isin(data.id)]
# y = y.loc[y.index.isin(data.id)]
# _,_,y_train,y_test = train_test_split(x,y,test_size=0.1)
# len(y_train),len(y_test)
# for sample in [1000,2000,5000,8000,10000,15000,20000]:
#     _ = y_train.to_frame().groupby('hotel_cluster').apply(lambda x: x.sample(frac=(sample/len(y_train)))).droplevel(level=0)
#     os.mkdir(f'../../pics_sample_{sample}')
#     for i in y_train.unique():
#         os.mkdir(f'../../pics_sample_{sample}/{i}')
        
#     for i,d in data.loc[data.id.isin(_.index)].iterrows():
#         cat = _.loc[_.index == d.id].values[0][0]
#         #print(d['path'].name)
#         name = d['path'].name
#         shutil.copy(d['path'],f'../../pics_sample_{sample}/{cat}/{name}')
        
# for i in y.unique():
#     os.mkdir(f'../../pics_test/{i}')        
    
# for i,d in data.loc[data.id.isin(y_test.index)].iterrows():
#     cat = y_test.loc[y_test.index == d.id].values[0]
#     #print(d['path'].name)
#     name = d['path'].name
#     shutil.copy(d['path'],f'../../pics_test/{cat}/{name}')

In [2]:
pos =  {
    'dense1':[32,64,173,128,256],
    'dense2':[64,128,256,512],
    'drop2':[0.1,0],
    'dense3':[64,128,256,512,1024],
    'drop3':[0.1,0,0.2],
    'dense4':[64,128,256,512,1024],
    'drop4':[0.1,0,0.2],
    'dense5':[64,128,256],
    'dense6':[64,128,256],
    'drop6':[0.1,0,0.2],
    'bn':[0,1]
    
}


In [6]:
for idx,params in enumerate(diff_params[5:]):
    print(params)

dense1
dense2
drop2
dense3
drop3
dense4
drop4
dense5
dense6
drop6
bn


In [2]:
diff_params = pd.read_pickle('../results/best_10_params_nn.pkl')

In [22]:
results = []
history = []  

params = diff_params.iloc[0].to_dict()
for sample_size in [1000,2000,5000,8000,10000,15000,20000,30000]:
    print(f'sample_size: {sample_size}')
    train_ds,val_ds = load_cv_data(sample_size)
    model = make_model_cv(params)
    h = model.fit(train_ds, batch_size=128, epochs=100, validation_data=val_ds, verbose = 0,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True,start_from_epoch=30)])
    
    h2  = {
        'val_loss':h.history['val_loss'],
        'val_accuracy':h.history['val_accuracy'],
        'loss':h.history['loss'],
        'accuracy':h.history['accuracy'],
        'sample_size' : sample_size,
    }
    
    history.append(h2.copy())
    print('tutaj')
    res = test_cv_model(model)
    res['sample_size'] = sample_size
    res['params_id'] = idx
    results.append(res)
    pd.DataFrame(results).to_pickle(f'../results/cv/res_sample_{sample_size}.pkl')
    pd.DataFrame(h2).to_pickle(f'../results/cv/hist_sample_{sample_size}.pkl')



sample_size: 1000
Found 1000 files belonging to 99 classes.
Using 700 files for training.
Using 300 files for validation.


KeyboardInterrupt: 

In [19]:
params

{'dense1': 128.0,
 'dense2': 512.0,
 'drop2': 0.1,
 'dense3': 128.0,
 'drop3': 0.0,
 'dense4': 128.0,
 'drop4': 0.0,
 'dense5': 64.0,
 'dense6': 64.0,
 'drop6': 0.0,
 'bn': 0.0}

In [15]:
model.build(input_shape=(30,30,30,30))
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_21 (Rescaling)    (173,)                    0         
                                                                 
 conv2d_63 (Conv2D)          multiple                  1355      
                                                                 
 max_pooling2d_63 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 conv2d_64 (Conv2D)          multiple                  460       
                                                                 
 max_pooling2d_64 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 conv2d_65 (Conv2D)          multiple                